## OpenStreet Map San Francisco California Financial District Data Case Study

I work in this area and wanted to do the study on this area to find what the OpenStreet data shows. 
The data was exported using the OpenStreetMap export tool (Linked below) by zooming in to the area of intrest and clicking on the Overpass API link

[San Francisco California Financial District](https://www.openstreetmap.org/export#map=15/37.7829/-122.4117)

---------------

### Understanding the map XML file and parrsing it using xml library in Python

Import required libraries for this study

In [11]:
#Import required libraries
import xml.etree.cElementTree as ET
from collections import defaultdict
import pprint
import re
import csv
import codecs
import schema
import sqlite3 
import pandas as pd

Use the iterative parsing to process the map file and
find what tags are there, and how many of each of those tags in the file, to get the
feeling on how much of which data you can expect to have in the map.

In [2]:
"""
Create a function called count_tags which iterate through the xml file looking for tags
and count those tags.
"""
def count_tags(file):
    tags = {} #create empty dic to hold values of tags and their counts
    for ev,elem in ET.iterparse(file):
        tag = elem.tag
        if tag not in tags.keys():
            tags[tag] = 1
        else:
            tags[tag]+=1
    return tags
count_tags('SF_MAP.xml')

{'note': 1,
 'meta': 1,
 'bounds': 1,
 'tag': 143144,
 'node': 235055,
 'nd': 292789,
 'way': 26374,
 'member': 81648,
 'relation': 1500,
 'osm': 1}

Check the "k" value for each tag and see if there are any potential problems.
count of each of

four tag categories in a dictionary:

- "lower", for tags that contain only lowercase letters and are valid,
- "lower_colon", for otherwise valid tags with a colon in their names,
- "problemchars", for tags with problematic characters, and
- "other", for other tags that do not fall into the other three categories.

In [3]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
            if lower.search(element.attrib['k']) != None: 
                keys['lower'] += 1
            elif lower_colon.search(element.attrib['k']) != None:
                keys['lower_colon'] += 1
            elif problemchars.search(element.attrib['k']) != None:
                keys['problemchars'] += 1
            else: 
                keys['other'] += 1
                
        
    return keys

def process_map(file):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(file):
        keys = key_type(element, keys)

    return keys
process_map('SF_MAP.xml')

{'lower': 104477, 'lower_colon': 37526, 'problemchars': 36, 'other': 1105}

Now let's find the number of users that contributed to this area of the map

In [4]:
def get_user(element):
    return

def process_map_user(file):
    users = set()
    for _, element in ET.iterparse(file):
        if element.tag == 'node' or element.tag == 'way' or element.tag == 'relation':
            users.add(element.attrib['user'])
    print("Total Contributors To This Map Area is:", len(users))
process_map_user('SF_MAP.xml')

Total Contributors To This Map Area is: 915


------------

### Data Auditing (Encoutered Problems)
First I will create an auditing function called audit_address which looks through the provided XML file and the content we are trying to audit and return the valuse that need auditing. Content can be street and postal code. 

In [5]:
def audit_address(filename, content):
        key='addr:'+ content
        expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons","Way", "Circle", "Key","Terrace", "Garden"]
    
# When the content is 'postcode', the function audits the validity of the postcode
# For our map we need zip codes that start with 94 
# Also for consistency only use 5 digits zip codes

        if content=='postcode':
            i=0   #set a records tracker 
            for event, elem in ET.iterparse(filename, events=("start",)):
                i+=1
                if elem.tag == "node" or elem.tag == "way":
                    for tag in elem.iter("tag"):
                        if tag.attrib['k']== key:
                            if (tag.attrib['v'][0:2]!='94') or (len(tag.attrib['v'])!=5): 
                                print(i)
                                print (tag.attrib['v'])

# When the content is 'street', the function audits the validity of the street name
# It returns a dictionary with street types as keys and corresponding counts as values. 

        elif content=='street':
            
            street_types = {}
            street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
            for event, elem in ET.iterparse(filename, events=("start",)):
                if elem.tag == "node" or elem.tag == "way":
                    for tag in elem.iter("tag"):
                        if tag.attrib['k']== key:
                            m = street_type_re.search(tag.attrib['v'])
                            if m:
                                street_type = m.group() #group(): Return the string matched by the RE
                                if street_type not in expected:
                                    street_types[street_type]=street_types.get(street_type,0)+1
            print (street_types)

**Findings From Auditing**

let's run the audit_address we created in the code snippet above for street names and postal codes.

In [6]:
# Zip codes that don't start with 94 or has more than 5 numbers
audit_address("SF_MAP.xml", "postcode")

5855
94103-3124
232094
90214
232099
90214
232104
90214
237763
95115
246389
94115-4620
266269
914105
610582
90214
610606
90214
610716
90214


As we can see above there are 20 zip codes that don't meet the specification of 5 digits or starts with 94

In [7]:
# Street Names
audit_address("SF_MAP.xml", "street")

{'Park': 7, 'Plaza': 8, 'Embarcadero': 17, 'St': 9, 'Center': 5, '730': 2, 'B': 1, 'A': 1, 'Building': 2, '3658': 1, 'Rock': 1, 'st': 2, 'Broadway': 41, 'Montgomery': 1, 'Hyde': 1, 'Post': 1, 'Kearny': 1, 'Pier': 1, 'Bldg': 1, 'St.': 3, 'Fell': 1, 'Ctr.': 1, '4.5': 1, 'North': 3, 'Alley': 5, 'Ave': 1, 'California': 1}


For the street auditing we can see the following that will need cleaninf before we move the data to a database:
- There different abbrevation for Street St, st, St.
- Building and Bldg refrence Buildings

In the next section we clean these qulaity issues and prpare the data for export into a database schema.

--------------

### Data Cleaning And Preparing

In this section we will clean the data and replace the mismatches we discussed in the Data audit section.

The code below will clean the data, and will use the function process_map to break the XML file into 5 different CSV files:
- nodes.csv
- nodes_tags.csv
- ways.csv
- ways_nodes.csv
- ways_tags.csv

Then read the data into these files based on the tag elements in the XML file.

In [23]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons","Way", "Circle", "Key","Terrace", "Garden"]

# These are the variables I will fix and what they are being changed to
mapping = { "St": "Street",
            "St.": "Street",
            "st": "Street",
           "Bldg": "Building"
            }

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

# Create Schema using the schema provided in the project instruction
SCHEMA = schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


#               Helper Functions            
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k : v for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


def update_name(name, mapping):
    street=street_type_re.search(name).group()

    name=name.replace(street, mapping[street])

    return name


#clean_element function take tag['value'] and tag['key'] as input and return the updated tag values 
def clean_element(tag_value, tag_key):
    
    ## clean postcode 
    if tag_key=='postcode':
        if (tag_value[0:2]!='94' and tag_value[0:2]!='95') or (len(tag_value)!=5):
            ## find postcode start with 'CA' and remove the 'CA' 
            if tag_value[0:2]=='CA': 
                    tag_value=tag_value[-5:]

    ## clean street suffix, change abbrivations to full street suffix        
    elif tag_key=='street':
        street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
        full_addr=tag_value
        m = street_type_re.search(full_addr)
        if m:
            street_type = m.group() #group(): Return the string matched by the RE
            if street_type not in expected:
                if street_type in mapping:
                    tag_value=update_name(full_addr, mapping) # call update_name function 
    ## return updated tag_value
    return tag_value
                             
## Clean and shape node or way XML element to Python dict

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=problemchars, default_tag_type='regular'):
   
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
   
    ## clean node element
    if element.tag=='node':
        for primary in element.iter():
            for i in node_attr_fields: 
                if i in primary.attrib: 
                    node_attribs[i]=primary.attrib[i]
        if len(element)!=0:
            for j in range(0, len(element)): 
                childelem=element[j]
                tag={}
                if not problem_chars.search(childelem.attrib['k']): ## ignor problematic element
                    tag["id"]=element.attrib["id"]
                    tag["type"]=default_tag_type
                    tag['value']=childelem.attrib['v']
                    if ":" in childelem.attrib['k']:
                        k_and_v=childelem.attrib['k'].split(':',1)
                        tag["type"]=k_and_v[0]
                        tag["key"]=k_and_v[1]
                        if tag["type"]=='addr':
                            tag["value"]=clean_element(tag["value"],tag["key"]) ## call clean_element function
                    else:
                        tag["key"]=childelem.attrib['k']
                        if tag["type"]=='addr':
                            print(tag_value, tag["key"])
                            tag["value"]=clean_element(tag["value"],tag["key"])
                tags.append(tag)
                
        return ({'node': node_attribs, 'node_tags': tags})            
                    
    ## handle way element               
    elif element.tag=='way':
        for primary in element.iter():
            for i in way_attr_fields: 
                if i in primary.attrib: 
                    way_attribs[i]=primary.attrib[i]   
        
        if len(element)!=0: 
            for j in range(0, len(element)): 
                childelem=element[j]
                tag={}
                if childelem.tag=='tag':
                    if not problem_chars.search(childelem.attrib['k']):
                        tag["id"]=element.attrib["id"]
                        tag["type"]=default_tag_type
                        tag["value"]=childelem.attrib['v']
                        if ":" in childelem.attrib['k']:
                            k_and_v=childelem.attrib['k'].split(':',1)
                            tag["key"]=k_and_v[1]
                            tag["type"]=k_and_v[0]
                            if tag["type"]=='addr':
                                tag["value"]=clean_element(tag["value"],tag["key"]) #call clean_element function
                        else:
                            tag["key"]=childelem.attrib['k']
                            if tag["type"]=='addr':
                                tag["value"]=clean_element(tag["value"],tag["key"]) #update tag values
                    tags.append(tag)
                    
                elif childelem.tag=='nd':
                    #print (childelem.attrib['ref'])
                    way_node={}
                    way_node['id']=element.attrib['id'] 
                    way_node['node_id']=childelem.attrib['ref']
                    way_node['position']=j
                    #print(way_node)
                    way_nodes.append(way_node)
                    
        return ({'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags})
    
## process the file, clean and write XML into csv according to given schema

def process_map(file_in):
    """Iteratively process each XML element and write to csv(s)"""
    with codecs.open(NODES_PATH, 'w', encoding='utf-8') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w', encoding='utf-8') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w', encoding='utf-8') as ways_file, \
        codecs.open(WAY_NODES_PATH, 'w', encoding='utf-8') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w', encoding='utf-8') as way_tags_file:
                
        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        
        nodes_writer.writeheader()
        
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()
    
       
        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])
                    
process_map("SF_MAP.xml")

### Creating sqllite Database

In [24]:
# ref https://stackoverflow.com/questions/50735349/import-csv-into-sqlite3-insert-failed
conn=sqlite3.connect('SF_MAP.db')
cur = conn.cursor() 
cur.execute("CREATE TABLE nodes ( id INTEGER PRIMARY KEY NOT NULL, lat REAL, lon REAL,\
    user TEXT, uid INTEGER, version INTEGER, changeset INTEGER, timestamp TEXT )")
conn.commit()
node_df = pd.read_csv('nodes.csv', dtype=object)
node_df.to_sql('nodes', conn, if_exists='append', index=False)


cur.execute("CREATE TABLE nodes_tags (\
    id INTEGER,\
    key TEXT,\
    value TEXT,\
    type TEXT,\
    FOREIGN KEY (id) REFERENCES nodes(id)\
)")
conn.commit()
nodetag_df=pd.read_csv('nodes_tags.csv')
nodetag_df.to_sql('nodes_tags', conn, if_exists='append', index=False)

cur.execute("CREATE TABLE ways (\
    id INTEGER PRIMARY KEY NOT NULL,\
    user TEXT,\
    uid INTEGER,\
    version TEXT,\
    changeset INTEGER,\
    timestamp TEXT\
)")
conn.commit()
way_df=pd.read_csv('ways.csv')
way_df.to_sql('ways', conn, if_exists='append', index=False)

cur.execute("CREATE TABLE ways_nodes (\
    id INTEGER NOT NULL,\
    node_id INTEGER NOT NULL, \
    position INTEGER NOT NULL, \
    FOREIGN KEY (id) REFERENCES ways(id),\
    FOREIGN KEY (node_id) REFERENCES nodes(id)\
)")
conn.commit()
waynode_df=pd.read_csv('ways_nodes.csv')
waynode_df.to_sql('ways_nodes', conn, if_exists='append', index=False)


cur.execute("CREATE TABLE ways_tags (\
    id INTEGER NOT NULL,\
    key TEXT NOT NULL,\
    value TEXT NOT NULL,\
    type TEXT,\
    FOREIGN KEY (id) REFERENCES ways(id)\
)")
conn.commit()
waytag_df=pd.read_csv('ways_tags.csv')
waytag_df=waytag_df.dropna(subset=['id', 'key', 'value'], how='any')
waytag_df.to_sql('ways_tags', conn, if_exists='append', index=False)

**Files Sizes**

In [42]:
# Ref https://stackoverflow.com/questions/574730/python-how-to-ignore-an-exception-and-proceed
# Ref https://stackabuse.com/python-list-files-in-a-directory/

import os

for root, dirs, files in os.walk("."):  
    try:
        for filename in files:
            print(filename, round(os.path.getsize(filename)/1000000, 2), 'MB')
    except:
        pass

nodes.csv 19.54 MB
nodes_tags.csv 1.82 MB
schema.py 0.0 MB
SF-MAP.ipynb 0.03 MB
SF_MAP.db 28.39 MB
SF_MAP.xml 57.9 MB
ways.csv 1.63 MB
ways_nodes.csv 7.09 MB
ways_tags.csv 2.9 MB


**Number of Unique Users**

In [48]:
query='''select count(DISTINCT uid) from nodes; '''

result=cur.execute(query)
for row in result:
    print (row)

(788,)


**Number of Nodes**

In [49]:
query='''select count(DISTINCT id) from nodes; '''

result=cur.execute(query)
for row in result:
    print (row)

(235055,)


**Number of Ways**

In [52]:
query='''select count(DISTINCT id) from ways; '''

result=cur.execute(query)
for row in result:
    print (row)

(26374,)


**Number of Bars in the area**

In [60]:
query='''select count(value) from nodes_tags where key = 'amenity' AND (value = 'bar' or value = 'pub'); '''

result=cur.execute(query)
for row in result:
    print (row)

(200,)


**Number of Place of Worship in the area**

In [58]:
query='''select count(value) from nodes_tags where key = 'amenity' AND value LIKE '%worship%'; '''

result=cur.execute(query)
for row in result:
    print (row)

(52,)


**Number of Schools**

In [62]:
query='''select count(value) from nodes_tags where key = 'amenity' AND value LIKE '%School%'; '''

result=cur.execute(query)
for row in result:
    print (row)

(30,)


### Suggestions And Improvements

Although OpenStreetMaps has guidance for the contributors to follow I suggest they have data validation rules when the data is being entered. The data validation will help with keeping the data consistent and will minimize errors. Also, there are many opportunities to bucket places into one category, for example, pubs and bars are separated in the amenity category but might be better to combine them into one, cinemas and theaters can also be combined.

This can help OpenStreetMaps to better manage the data, which will also take up less space and will give the users less options to choose from when tagging locations

Below query shows all ameneties and how we can bucket some of the sub categories into one:

In [67]:
query='''select DISTINCT value from nodes_tags where key = 'amenity'; '''

result=cur.execute(query)
for row in result:
    print (row)

('pub',)
('bar',)
('post_office',)
('parking_entrance',)
('cinema',)
('fuel',)
('ferry_terminal',)
('parking',)
('post_box',)
('cafe',)
('nightclub',)
('drinking_water',)
('toilets',)
('bicycle_parking',)
('restaurant',)
('telephone',)
('library',)
('fast_food',)
('bank',)
('school',)
('place_of_worship',)
('university',)
('fountain',)
('language_school',)
('college',)
('nursing_home',)
('theatre',)
('ice_cream',)
('arts_centre',)
('fire_station',)
('public_building',)
('police',)
('atm',)
('taxi',)
('car_rental',)
('brokerage',)
('car_sharing',)
('clock',)
('gym',)
('community_centre',)
('pharmacy',)
('dentist',)
('salon',)
('hookah_lounge',)
('club',)
('bench',)
('vending_machine',)
('event',)
('social_centre',)
('social_facility',)
('kindergarten',)
('carpool',)
('clinic',)
('waste_basket',)
('stripclub',)
('footpath',)
('doctors',)
('marketplace',)
('shelter',)
('recycling',)
('embassy',)
('coworking_space',)
('conference_centre',)
('bicycle_rental',)
('vintage and modern resale',)